In [1]:
import pandas as pd
import datetime
import numpy as np
from nombres import normalizacion_nombres
from nacionalidades import normalizacion_paises
from institucion import normalizacion_universidades
from funciones import limpiar_caracteres_especiales, limpiar_funciones, instrumento_financiamiento, normalizacion_ciudad, ciudad_a_region
from funciones import institucion_instrumento, descripcion_codigo_inach, region_institucion, macrozona_region, proyecto_concursable
from funciones import categorizacion_fondos_wos, filtrar_normalizacion_funding_wos
from area_ocde import equivalencia_objetivos

In [2]:
ruta_lectura = "wos/chile/"
ruta_final = "wos/chile/finales/"
lista_hojas = [1, 2]

### Trabajos WOS

In [3]:
tmp_xls = []
dict_orcids = dict()

for n in lista_hojas:
    xls_base = pd.read_excel(f"{ruta_lectura}chile-2010-2023({n}).xls")
    for i,row in enumerate( xls_base.to_numpy().tolist() ):
        authors_short = [ n.lower().strip().split(',') for n in row[0].split(';') ]
        authors_short = [limpiar_caracteres_especiales( f'{nom.strip()} {ape.strip()}' ) for ape, nom in authors_short]
        
        authors_full = [ n.lower().strip().split(',') for n in row[1].split(';') ]
        authors_full = [limpiar_caracteres_especiales( f'{nom.strip()} {ape.strip()}' ) for ape, nom in authors_full]
        source_title = limpiar_caracteres_especiales( row[3].lower().strip() )
        
        orcid = [ p.lower().strip() for p in row[14].split(';') ]
        orcid = [ p.split('/') for p in orcid if p!='noinf']
        for p, o in orcid:
            if p == '': p= "noinf, noinf"
            ape, nom = p.lower().strip().split(',')
            nombre = limpiar_caracteres_especiales( f'{nom.strip()} {ape.strip()}' )
            dict_orcids[ nombre ] = o

        funding = row[15]
        funding = funding.split(';')
        funding = [ categorizacion_fondos_wos(f) for f in funding ]
        anid, inach, extranjero, corfo, otro = filtrar_normalizacion_funding_wos( funding )

        year = row[30]
        doi = row[31]
        address = row[9]
        affiliation = row[10]

        #for person_full, person_short in zip(authors_full, authors_short):
        #    pass
            #if person_full in dict_orcids.keys():
            #    print( dict_orcids[person_full] )
        tmp_xls.append([source_title, doi, year, anid, inach, extranjero, corfo, otro])

### Personas WOS